In [ ]:
import chromadb
client = chromadb.PersistentClient(path="./db")

In [ ]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        return embedding

In [ ]:
import os

collection = client.get_or_create_collection(name="chromadb_demo2_hf_model", embedding_function=MyEmbeddingFunction(api_key=os.getenv("HF_API_KEY"), enpoint_url=os.getenv("EMBEDDED_ENDPOINT")))

In [ ]:
import csv

def get_csv_file(filename):
    # Read the data from the CSV file
    with open(filename, "r") as f:
        # Skip the header row
        next(f)
        reader = csv.reader(f)
        return list(reader)

# Get the data from the CSV file
data = get_csv_file("chinese_menu_items.csv")



In [ ]:
# Add the data to the collection
collection.add(
    ids=[arr[0] for arr in data],
    documents=[arr[1] for arr in data],
    metadatas=[{"cuisine":"chinese"} for _ in data]
)

In [7]:
chunk_size = 25

# Flatten the data into two lists
ids=[arr[0] for arr in data]
docs=[arr[1] for arr in data]

# Split the data into chunks
id_chunks = [ids[i:i + chunk_size] for i in range(0, len(ids), chunk_size)]
doc_chunks = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

In [8]:
# Add the data to the collection

for id_chunk, doc_chunk in zip(id_chunks, doc_chunks):
    collection.add(
        ids=id_chunk,
        documents=doc_chunk,
        metadatas=[{"cuisine":"chinese"} for _ in id_chunk]
    )

In [9]:
result = collection.query(
    query_texts=["greasy noodles"],
    n_results=5,
    include=["documents"],
    where={"cuisine": "chinese"}
)

result["documents"]

[['Mixed Vegetable Pan Fried Noodle',
  'Pan Fried Noodle with Tofu and Curry Sauce',
  'Mixed Vegetable and Tofu Pan Fried Noodle',
  'Soy Sauce Chow Mein',
  'Braised Noodle']]